In [1]:
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, Dense, Lambda
import keras.applications.mobilenet_v2 as mobilenetv2
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [2]:
# Load MobileNetV2 base model without the top layer
mobilenetv2_layer = mobilenetv2.MobileNetV2(include_top=False, 
                                            input_shape=(224, 224, 3),
                                            weights='../mobilenet-v2-keras-weights/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5')

mobilenetv2_layer.trainable = False

In [3]:
# Create the model architecture
model = Sequential()
# Input layer
model.add(tf.keras.Input(shape=(224, 224, 3)))

# Lambda preprocessing layer
def mobilenetv2_preprocessing(img):
    return mobilenetv2.preprocess_input(img)

model.add(Lambda(mobilenetv2_preprocessing))

model.add(mobilenetv2_layer)

# Add the GlobalAveragePooling2D layer
model.add(GlobalAveragePooling2D())

# Update the Dense layer for 2 classes
model.add(Dense(2, activation='softmax'))

# Compile the model with a lower learning rate for fine-tuning
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=1e-4), metrics=['categorical_accuracy'])

# Print the updated model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda (Lambda)             (None, 224, 224, 3)       0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 2)                 2562      
                                                                 
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [4]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping

In [5]:
# train_dir = '../../trainingData/'
train_dir = 'trainingData/'

In [6]:
# Create an ImageDataGenerator with data augmentation and rescaling
train_datagen = ImageDataGenerator(
    # rescale=1./255,
    # zoom_range=0.2,
    # rotation_range=5,
    # brightness_range=[0.9, 1.1],
    # horizontal_flip=True,
    # vertical_flip=True,
    validation_split=0.1
)

# Load training data with flow_from_directory
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),      # Resizes images to 224x224
    batch_size=16,               # Number of images per batch
    class_mode='categorical', 
    subset='training'
)

# Load validation data
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    subset='validation'
)

# Print class indices to verify the labels
print(f"Class Indices: {train_generator.class_indices}")

Found 324 images belonging to 2 classes.
Found 36 images belonging to 2 classes.
Class Indices: {'metal': 0, 'plastic': 1}


In [65]:
class_weights = {0: 1.0, 1: 1.2}

In [7]:
early_stop = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=5,          # Stop after 5 epochs of no improvement
    restore_best_weights=True  # Restore the best weights
)

In [8]:
# Fine-tune the model on the new data
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[early_stop],
    # class_weight=class_weights,
    epochs=15
)

Epoch 1/15
20/20 [==============================] - 6s 202ms/step - loss: 0.6738 - categorical_accuracy: 0.6071 - val_loss: 0.4888 - val_categorical_accuracy: 0.8438
Epoch 2/15
20/20 [==============================] - 3s 167ms/step - loss: 0.5154 - categorical_accuracy: 0.7630 - val_loss: 0.3759 - val_categorical_accuracy: 0.9375
Epoch 3/15
20/20 [==============================] - 3s 158ms/step - loss: 0.4131 - categorical_accuracy: 0.8734 - val_loss: 0.2967 - val_categorical_accuracy: 0.9062
Epoch 4/15
20/20 [==============================] - 3s 157ms/step - loss: 0.3491 - categorical_accuracy: 0.9058 - val_loss: 0.2368 - val_categorical_accuracy: 0.9375
Epoch 5/15
20/20 [==============================] - 3s 161ms/step - loss: 0.2907 - categorical_accuracy: 0.9469 - val_loss: 0.2261 - val_categorical_accuracy: 0.9062
Epoch 6/15
20/20 [==============================] - 3s 156ms/step - loss: 0.2470 - categorical_accuracy: 0.9545 - val_loss: 0.1946 - val_categorical_accuracy: 0.9375
Epoc

In [9]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {loss}, Validation Accuracy: {accuracy}")

# Save the fine-tuned model
model.save('fine_tuned_plastic_metal_model.h5')

3/3 [==============================] - 0s 96ms/step - loss: 0.0825 - categorical_accuracy: 1.0000
Validation Loss: 0.08247848600149155, Validation Accuracy: 1.0


### Inference

In [10]:
from tensorflow.keras.models import load_model
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the fine-tuned model
model = load_model('fine_tuned_plastic_metal_model.h5')

In [17]:
# img_path = '../sample_data/metal_coke.jpg'
# img_path = '../sample_data/plastic_bottle_out_of_shape.jpg'
# img_path = 'trainingData/metal/image_20240922_040431_4.jpg'
# img_path = 'trainingData/metal/image_20240922_034123_3.jpg'
img_path = 'trainingData/plastic/image_20240922_023123_10.jpg'
# img_path = 'trainingData/plastic/image_20240922_025322_17.jpg'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = mobilenetv2.preprocess_input(img_array)

In [18]:
# Predict using the fine-tuned model
predictions = model.predict(img_array)
print(predictions)
class_index = np.argmax(predictions, axis=1)

# Map the predicted index to class label
class_labels = ['metal', 'plastic']
predicted_class = class_labels[class_index[0]]
print(f'Predicted Class: {predicted_class}')

1/1 [==============================] - 0s 28ms/step
[[0.5467155  0.45328447]]
Predicted Class: metal
